# 1、LLMChain的使用

举例1：

In [2]:
from modulefinder import replacePackageMap

from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SimpleSequentialChain
from langchain_core.prompts import PromptTemplate

import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 1、创建大模型实例
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 2、提供提示词模板
# BasePromptTemplate的典型子类有：PrompTemplate、ChatPromptTemplate
prompt_template = PromptTemplate.from_template(
    template="你是一个数学高手，帮我解决如下的数学问题：{question}"
)

chain = LLMChain(llm=chat_model, prompt=prompt_template, )
response = chain.invoke(input={"question": "1 + 2 * 3 = ?"})
print(response)

{'question': '1 + 2 * 3 = ?', 'text': '根据数学运算的优先级，我们首先要进行乘法运算，然后再进行加法运算。\n\n所以，计算过程如下：\n\n1 + 2 * 3 = 1 + 6 = 7\n\n因此，1 + 2 * 3 = 7。'}


举例2：使用ChatPromptTemplate及参数verbose的演示

In [3]:
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 1、创建大模型实例
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 2、提供提示词模板
# BasePromptTemplate的典型子类有：PrompTemplate、ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个数学高手"),
    ("human", "帮我解决如下的数学问题：{question}")

])

chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template,
    verbose=True,  #显式执行过程中的详细日志情况
)
response = chain.invoke(input={"question": "1 + 2 * 3 = ?"})
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个数学高手
Human: 帮我解决如下的数学问题：1 + 2 * 3 = ?

> Finished chain.
{'question': '1 + 2 * 3 = ?', 'text': '根据数学运算的优先级，先进行乘法再进行加法。因此我们先计算 \\( 2 * 3 \\)，得到 \\( 6 \\)。然后再进行加法：\n\n\\[ \n1 + 6 = 7 \n\\]\n\n所以 \\( 1 + 2 * 3 = 7 \\)。'}


# 2、顺序链之SimpleSequentialChain的使用

举例1：


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

chainA_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位精通各领域知识的知名教授"),
        ("human", "请你尽可能详细的解释一下：{knowledge}"),
    ]
)

chat_model = ChatOpenAI(model="gpt-4o-mini")

chainA_chains = LLMChain(llm=chat_model,
                         prompt=chainA_template,
                         verbose=True
                         )



# chainA_chains.invoke({"knowledge":"什么是LangChain？"})

In [6]:
from langchain_core.prompts import ChatPromptTemplate

chainB_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
        ("human", "这是针对一个提问的完整的解释说明内容：{description}"),
        ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在20个字以内"),
    ]
)

chainB_chains = LLMChain(llm=chat_model,
                         prompt=chainB_template,
                         verbose=True
                         )

In [9]:
from langchain.chains.sequential import SimpleSequentialChain

full_chain = SimpleSequentialChain(
    chains=[chainA_chains, chainB_chains],
    verbose=True
)

#说明：针对于SimpleSequentialChain而言，唯一的输入的变量名是：input
response = full_chain.invoke(input={"input": "什么是LangChain?"})
print(response)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你尽可能详细的解释一下：什么是LangChain?

> Finished chain.
LangChain 是一个开源的框架，旨在帮助开发者构建基于大语言模型（LLM, Large Language Models）和其他 AI 组件的应用程序。LangChain 的设计初衷是通过提供丰富的工具和结构，简化与语言模型的整合和应用开发过程，让开发者能够更高效地构建复杂的 NLP（自然语言处理）应用。

### LangChain 的核心概念

1. **链（Chains）**：
   LangChain 的核心概念是“链”，即将一个或多个操作或组件组合在一起，按顺序执行。每个链可以是简单的，也可以是复杂的，允许开发者定义输入和输出的关系。链可以包括查询、转换、验证等多个步骤。

2. **代理（Agents）**：
   代理是一种可动态决定其行为和路径的结构，它使用一个语言模型来理解用户输入，并选择最合适的行动。代理通常在需要实时决策的场景下使用，比如与用户的对话。

3. **内存（Memory）**：
   LangChain 提供了内存管理功能，允许应用在不同的会话中保留用户的上下文和历史信息。这样可以使得对话系统能够更好地理解用户的需求和偏好。

4. **工具（Tools）**：
   LangChain 支持将外部工具和 API 集成到链或代理中，使得语言模型能够进行逻辑推理、查询数据库、调用 web 服务等、并处理更复杂的任务。

5. **文档（Documents）**：
   文档处理是 LangChain 另一个重要功能。在需要处理大量文本信息的场景下，LangChain 提供了文档检索和分析的能力，以支持信息提取、总结等任务。

### LangChain 的使用场景

LangChain 可以用于多种应用场景，包括但不限于：

- **聊天机器人**：通过构建链和代理，开发者可以创建能够与用户进行自然交互的对话系统。
- **内容生成**：LangChain 可以帮

举例2：


In [10]:
# 1.导入相关包
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

# 2.创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# 3.定义一个给剧名写大纲的LLMChain
template1 = """你是个剧作家。给定剧本的标题，你的工作就是为这个标题写一个大纲。
Title: {title}
"""
prompt_template1 = PromptTemplate(input_variables=["title"], template=template1)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template1)

# 4.定义给一个剧本大纲写一篇评论的LLMChain
template2 = """你是《纽约时报》的剧评家。有了剧本的大纲，你的工作就是为剧本写一篇评论
剧情大纲:
{synopsis}
"""
prompt_template2 = PromptTemplate(input_variables=["synopsis"], template=template2)
review_chain = LLMChain(llm=llm, prompt=prompt_template2)


# 5.定义一个完整的链按顺序运行这两条链
#(verbose=True:打印链的执行过程)
overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain],
    verbose=True
)
# 6.调用完整链顺序执行这两个链
review = overall_chain.invoke({"input":"日落海滩上的悲剧"})

# 7.打印结果
print(review)



> Entering new SimpleSequentialChain chain...
标题: 日落海滩上的悲剧

大纲:

**第一幕：初见与冲突**
- 场景设定在一个风景如画的海滩小镇，黄昏时分，阳光洒在波浪上，营造出宁静的氛围。
- 主要角色介绍：艾米莉（年轻的艺术家），她正在海滩上寻求灵感；杰克（早年回乡的海军军人），内心承受着战争的创伤；老渔夫哈罗德（镇上的智者，为人和蔼却满腹辛酸）。
- 艾米莉偶然遇见杰克，两人因对海滩的不同看法而产生争执。杰克认为战争是人类无法逃避的悲剧，而艾米莉则认为艺术能治愈创伤。

**第二幕：友谊与秘密**
- 在一次海滩晚会中，艾米莉和杰克逐渐和解并成为朋友。在哈罗德的引导下，他们分享各自的经历，建立起深厚的情感纽带。
- 艾米莉对杰克的过去产生了浓厚的兴趣并尝试用画笔捕捉他的故事，然而，杰克对战争的回忆仍然充满痛苦，他试图隐藏自己内心的黑暗。
- 镇上发生了一件神秘的事情：一位年轻女孩在海滩失踪，镇子陷入恐慌，杰克的内心冲突加剧。

**第三幕：真相揭示与选择**
- 艾米莉和杰克决定展开调查，试图找出女孩失踪的真相。在这个过程中，他们发现了小镇隐藏的秘密，揭示了人们对于过去的逃避以及对现实的无力。
- 在一次深夜探险中，艾米莉在海滩发现了旧日战争留下的遗物，勾起了杰克痛苦的记忆，最终他在海滩上爆发出情绪，揭示了自己在战争中亲历的悲惨事件。
- 二人面对各自的伤痛，杰克决定向镇上人们讲述真相，而艾米莉则决定用艺术记录下他们的故事。

**第四幕：悲剧与救赎**
- 当杰克表白自己的经历后，镇上的人们被震撼，部分人开始意识到直面过去的重要性。
- 然而，就在大家逐渐适应这个真相时，新的悲剧发生：杰克因无法承受内心的重负而选择极端方式结束自己的生命。艾米莉在此刻感受到深深的绝望与悔恨。
- 最终，在杰克的葬礼上，艾米莉发表演讲，向小镇的居民呼吁珍惜生命与彼此，她决定以画作纪念杰克，把他的故事传递下去。

**结尾：希望与重生**
- 画展上的艾米莉将杰克的故事以艺术的形式展现出来，吸引了越来越多人的关注。
- 海滩夕阳再次落下，艾米莉站在海滩上，手中握着画笔，她意识到，虽然生命充满悲剧，但每个人的故事都有被听见的价值，同时也渴望着希望与救赎。
**剧评：《日落海滩上的悲剧》——在艺术与痛苦之间的抉择**

《日落海滩

# 3、顺序链之SequentialChain的使用

举例1：


In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from openai import OpenAI
import os

# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")


schainA_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位精通各领域知识的知名教授"),
        ("human", "请你先尽可能详细的解释一下：{knowledge}，并且{action}")
    ]
)

schainA_chains = LLMChain(llm=llm,
                          prompt=schainA_template,
                          verbose=True,
                          output_key="schainA_chains_key"
                          )

# schainA_chains.invoke({
#     "knowledge": "中国的篮球怎么样？",
#     "action": "举一个实际的例子"
# }
# )

schainB_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
        ("human", "这是针对一个提问完整的解释说明内容：{schainA_chains_key}"),
        ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在100个字以内"),
    ]
)

schainB_chains = LLMChain(llm=llm,
                         prompt=schainB_template,
                         verbose=True,
                         output_key='schainB_chains_key'
                        )

# 一定要声明出两个变量：input_variables、output_variables
Seq_chain = SequentialChain(
                            chains=[schainA_chains, schainB_chains],
                            input_variables=["knowledge", "action"],
                            output_variables=["schainA_chains_key","schainB_chains_key"],
                            verbose=True)

response = Seq_chain.invoke({
                       "knowledge":"中国足球为什么踢得烂",
                       "action":"举一个实际的例子"
                    }
                )

print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你先尽可能详细的解释一下：中国足球为什么踢得烂，并且举一个实际的例子

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 你非常善于提取文本中的重要信息，并做出简短的总结
Human: 这是针对一个提问完整的解释说明内容：中国足球之所以在国际舞台上表现不佳，原因复杂且多方面，主要可以从以下几个方面进行分析：

### 1. **体制与管理问题**
   - **行政干预**：中国足球的管理体制存在较多行政干预，导致俱乐部和球队的发展受到限制。足球协会的决策往往受到政府部门的影响，缺乏独立性和专业性。
   - **腐败现象**：历史上，中国足球曾暴露出不少腐败问题，包括假球和贿赂，损害了运动员的职业道德和观众的信任。

### 2. **青训体系不完善**
   - **基础设施不足**：尽管近年来中国在足球场地建设上投入较多，但青训体系和基础设施的建设仍然滞后于其他足球强国。
   - **青少年培养**：在青少年足球领域，缺乏系统的培训和选拔机制，优秀的青少年球员难以脱颖而出并得到良好的培养。

### 3. **专业化水平欠缺**
   - **教练水平**：虽然一些外籍教练加入了中超联赛，但整体而言，中国本土教练的专业水平和战术素养较低，无法有效提升球队的战斗力。
   - **运动员素质**：由于缺乏高水平的比赛经验和科学的训练方法，中国足球运动员在技术、战术理解和身体素质等方面相对薄弱。

### 4. **文化与氛围**
   - **足球文化缺失**：在中国，足球文化相对薄弱，缺乏对足球的深厚热爱和支持，球迷基础较为薄弱，职业联赛的氛围不如一些足球强国。
   - **社会压力**：踢足球往往被视为不务正业，家长和社会对运动员的期望集中在学业和职业安全，导致年轻人对足球的追求不足。

### 5. **球员流动性与联赛质量**
   - **外援政策**：中超联赛吸

举例2：

In [14]:
# 1.导入相关包
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import SequentialChain

# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# 2.定义任务链一
#chain 1 任务：翻译成中文
first_prompt = PromptTemplate.from_template("把下面内容翻译成中文:\n\n{content}")
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=True,
    output_key="Chinese_Review",
)

# 3.定义任务链二
#chain 2 任务：对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = PromptTemplate.from_template("用一句话总结下面内容:\n\n{Chinese_Review}")
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    verbose=True,
    output_key="Chinese_Summary",
)

# 4.定义任务链三
# chain 3 任务：识别语言
third_prompt = PromptTemplate.from_template("下面内容是什么语言:\n\n{Chinese_Summary}")
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    verbose=True,
    output_key="Language",
)

# 5.定义任务链四
#chain 4 任务:针对摘要使用指定语言进行评论 input_key是上一个chain的output_key
fourth_prompt = PromptTemplate.from_template("请使用指定的语言对以下内容进行评论:\n\n内容:{Chinese_Summary}\n\n语言:{Language}")
chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    verbose=True,
    output_key="Comment",
)


# 6.总链
#overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言进行评论
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Review", "Chinese_Summary", "Language", "Comment"],
)


#读取文件
# read file
content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com)."
response = overall_chain.invoke(content)
print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
把下面内容翻译成中文:

Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
用一句话总结下面内容:

最近，我们迎来了几位新团队成员，他们在各自部门中做出了重要贡献。我想特别表扬简·史密斯（社会安全号：049-45-5928）在客户服务方面的杰出表现。简始终获得客户的积极反馈。此外，请记得我们的员工福利计划的开放报名期即将到来。如果您有任何问题或需要帮助，请联系我们的HR代表迈克尔·约翰逊（电话：418-492-3850，电子邮件：michael.johnson@example.com）。

> Finished chain.

In [15]:
print(response["Comment"])

这段内容简洁明了，强调了对新团队成员的欢迎和对表现优异员工的表扬，体现了团队的积极氛围和凝聚力。同时，提醒大家关注即将到来的员工福利计划报名期，显示出公司对员工福利的重视，有助于增强员工的归属感和参与感。整体来看，这是一则鼓舞人心且有实用价值的通知。
